In [24]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error


## Preprocessing

In [25]:
# Read
housing_data = pd.read_csv("housing.csv")
# Fill NA
housing_data['total_bedrooms'] = housing_data['total_bedrooms'].fillna(
    (housing_data['total_bedrooms'].sum())/len(housing_data['total_bedrooms']))
# Clean targets
housing_data = housing_data[housing_data['median_house_value'] < (housing_data['median_house_value'].max())]
# Encode categories and replace columns
housing_cat = housing_data['ocean_proximity']
housing_cat_encoded,housing_categories = housing_cat.factorize()
encoder = OneHotEncoder()
housing_cat_1hot = encoder.fit_transform(housing_cat_encoded.reshape(-1,1))
housing_cat_1hot = housing_cat_1hot.toarray()
ocean_proximity_cat_1hot = pd.DataFrame(housing_cat_1hot)
ocean_proximity_cat_1hot = ocean_proximity_cat_1hot.rename(columns=
                            {0: 'NEAR BAY', 1: '<1H OCEAN', 2:'INLAND', 3:'NEAR OCEAN', 4:'ISLAND'})
housing_data = housing_data.reset_index(drop=True)
housing_data = pd.concat([housing_data, ocean_proximity_cat_1hot], axis=1, sort=False)
housing_data = housing_data.drop('ocean_proximity',axis=1)
# Train test split
housing_train, housing_test = train_test_split(housing_data, test_size=0.33, random_state=11)


Needs to be edited/removed

In [26]:
class Tree :
    def __init__ (self,X_train,y_train,X_test,y_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        
    def fit_tree (self,depth):
        dTree = DecisionTreeRegressor(max_depth=depth)
        return dTree.fit(self.X_train,self.y_train)
    
    def predict_tree (self,depth):
        pred = self.fit_tree(depth).predict(self.X_test)
        return pred
    
    def evaluate_tree (self,depth):
        mse = mean_squared_error(self.y_test,self.predict_tree(depth))
        rmse = np.sqrt(mse)
        return rmse      

## Decision Tree Regression:
## Functions

In [166]:
depth = 2
X_train = housing_train[['latitude','median_income']]
y_train = housing_train[['median_house_value']]

my_tree = DecisionTreeRegressor(max_depth=depth)
my_tree.fit(X_train,y_train)
#pred = my_tree.predict(X_train)

tree_array = [my_tree]
num_nodes = my_tree.tree_.capacity
index = 0
alpha = 0
k = 1

tree_array[0].tree_.n_node_samples


array([13182,  8767,  4399,  4368,  4415,  3114,  1301])

In [198]:
def leaf_impurity(tree, index):
    if tree.tree_.children_left[index] == _tree.TREE_LEAF:   
            return (tree.tree_.n_node_samples[index] *  tree.tree_.impurity[index])
        
    
subtree_1 = leaf_impurity(tree_array[0],2)+leaf_impurity(tree_array[0],3)
subtree_4 = leaf_impurity(tree_array[0],5)+leaf_impurity(tree_array[0],6)
print(subtree_1)
print(subtree_4)
#
#if tree.tree_.children_left[index] != _tree.TREE_LEAF:
#            impurity_left, leafs_left = leaf_impurity(tree, tree.tree_.children_left[index])
#            impurity_right, leafs_right = leaf_impurity(tree, tree.tree_.children_right[index])
#            print ('Leaf ' + str(index) + ' Impurity: ' + str(impurity_left+impurity_right), 
#                   'composed of ' + str(leafs_left+leafs_right) + ' leaves')                 

48082470140780.516
29038658908520.07


In [170]:

print "Children left side: " + str(tree_array[0].tree_.children_left)
print "Tree n node samples: " + str(tree_array[0].tree_.n_node_samples) 
print "Tree Impurity: " + str(tree_array[0].tree_.impurity)

        

#(77121129049300.6, 4)
#(48082470140780.516, 2)
#(29038658908520.07, 2)




Children left side: [ 1  2 -1 -1  5 -1 -1]
Tree n node samples: [13182  8767  4399  4368  4415  3114  1301]
Tree Impurity: [9.49045916e+09 6.31294054e+09 4.56808725e+09 6.40738423e+09
 8.27535008e+09 6.57003518e+09 6.59459596e+09]


In [162]:
while num_nodes > 1:
    tree_array.append(copy.deepcopy(tree_array[k - 1]))

    min_node_idx, min_gk = determine_alpha(tree_array[k].tree_)

    prune(tree_array[k].tree_, min_node_idx)

    num_nodes = sum(1 * (tree_array[k].tree_.n_node_samples != 0))

    k += 1


    if False:
        for k in range(0,len(tree_array)):
            export_graphviz(tree_array[k], out_file='tree' + str(k) + '.dot')

('IDX: ', 0, 'NI: ', 125103232624977.56, 'STI: ', 77121129049300.6, 'STL: ', 4)
('IDX: ', 1, 'NI: ', 55345549705438.414, 'STI: ', 48082470140780.516, 'STL: ', 2)
('IDX: ', 4, 'NI: ', 36535670619936.86, 'STI: ', 29038658908520.07, 'STL: ', 2)
('IDX: ', 0, 'NI: ', 125103232624977.56, 'STI: ', 84384208613958.48, 'STL: ', 3)
('IDX: ', 4, 'NI: ', 36535670619936.86, 'STI: ', 29038658908520.07, 'STL: ', 2)
('IDX: ', 0, 'NI: ', 125103232624977.56, 'STI: ', 91881220325375.28, 'STL: ', 2)


In [ ]:
def _calc_impurity(tree, index):
    """
    Calc_impurity is a recursive function for calculating the absolute impurity of any subtree.
    The absolute impurity is calculated by the impurity of every leaf-node scaled with the number of samples per node.
    @:param tree: sk-learn tree object
    @:param index: the index of the root node of the subtree
    @:returns impurity and leaf count of subtree
    """

    # print("index: ", index, " impurity: ", d_tree.tree_.n_node_samples[index] * tree.impurity[index] / 10000000)
    # wenn es 'children' besuche die 'children'
    if tree.children_left[index] != _tree.TREE_LEAF:
        impurity_left, leafs_left = _calc_impurity(tree, tree.children_left[index])
        impurity_right, leafs_right = _calc_impurity(tree, tree.children_right[index])

        return impurity_left + impurity_right, leafs_left + leafs_right

    # wenn es keine 'children' gibt bin ich ein leaf Knoten
    else:
        # print("index: ", index, " cost: ", d_tree.tree_.n_node_samples[index] * tree.impurity[index]/10000000)
        return tree.n_node_samples[index] * tree.impurity[index], 1



In [92]:
print _calc_impurity(tree=my_tree.tree_,index=0)
print _calc_impurity(tree=my_tree.tree_,index=1)
print _calc_impurity(tree=my_tree.tree_,index=2)
print _calc_impurity(tree=my_tree.tree_,index=3)
print _calc_impurity(tree=my_tree.tree_,index=4)
print _calc_impurity(tree=my_tree.tree_,index=5)
print _calc_impurity(tree=my_tree.tree_,index=6)


(77121129049300.6, 4)
(48082470140780.516, 2)
(20095015826943.496, 1)
(27987454313837.02, 1)
(29038658908520.07, 2)
(20459089565783.688, 1)
(8579569342736.382, 1)


In [30]:
import sys
from sklearn.tree import _tree

def prune(inner_tree, index):
    """
    The given tree object is modified to resemble a version pruned at index
    @:param inner_tree: sk-learn tree object
    @:param index: the index of the node at which inner_tree is pruned
    """

    # wenn es 'children' gibt besuche ich die 'children'
    if inner_tree.children_left[index] != _tree.TREE_LEAF:
        prune(inner_tree, inner_tree.children_left[index])

        prune(inner_tree, inner_tree.children_right[index])

        # set node to leaf
        idx_left = inner_tree.children_left[index]
        idx_right = inner_tree.children_right[index]

        inner_tree.children_left[index] = _tree.TREE_LEAF
        inner_tree.children_right[index] = _tree.TREE_LEAF

        inner_tree.n_node_samples[idx_left] = 0
        inner_tree.n_node_samples[idx_right] = 0

        inner_tree.impurity[idx_left] = 0
        inner_tree.impurity[idx_right] = 0


    else:
        # wenn es keine 'children' gibt kann ich prunen
        inner_tree.n_node_samples[index] = 0


def determine_alpha(tree):
    """
    Given a regression tree, the relevant penalty scalars gk are determined for pruning. Every
    inner node of the tree is visit to evaluate the penalty scalar gk that would make pruning in each node reasonable.
    The minimum gk is returned
    @:param tree: sk-learn tree object
    @:returns the index and corresponding values of the minimal gk found in the tree (alpha)
    """
    min_gk = sys.maxsize
    min_node_idx = tree.node_count

    # traverse all inner nodes in to find min_gk
    for node_idx in range(tree.node_count):
        # if node is a leaf node, skip node
        if tree.children_left[node_idx] == _tree.TREE_LEAF:
            continue

        # inner node
        node_impurity = tree.n_node_samples[node_idx] * tree.impurity[node_idx]
        subtree_impurity, subtree_leafs = _calc_impurity(tree, node_idx)

        print('IDX: ', node_idx, 'NI: ', node_impurity, 'STI: ', subtree_impurity, 'STL: ', subtree_leafs)

        gk = (node_impurity - subtree_impurity) / (subtree_leafs - 1.)

        if gk < min_gk:
            min_node_idx = node_idx
            min_gk = gk

    return min_node_idx, min_gk




In [33]:
import copy

import pandas as pd
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor

#from classifier.prune import determine_alpha, prune

sns.set()

if __name__ == '__main__':
    data = pd.read_csv("housing.csv")

    input_features = data[['latitude', 'longitude']]
    target = data['median_house_value'] / 100000

    d_tree = DecisionTreeRegressor(max_depth=2)
    d_tree.fit(input_features, target)

    tree_array = [d_tree]

    num_nodes = d_tree.tree_.capacity
    index = 0
    alpha = 0
    k = 1

    while num_nodes > 1:
        tree_array.append(copy.deepcopy(tree_array[k - 1]))

        min_node_idx, min_gk = determine_alpha(tree_array[k].tree_)

        prune(tree_array[k].tree_, min_node_idx)

        num_nodes = sum(1 * (tree_array[k].tree_.n_node_samples != 0))

        k += 1


    if False:
        for k in range(0,len(tree_array)):
            export_graphviz(tree_array[k], out_file='tree' + str(k) + '.dot')

('IDX: ', 0, 'NI: ', 27483.198193687535, 'STI: ', 23353.654436952438, 'STL: ', 4)
('IDX: ', 1, 'NI: ', 23794.915979009485, 'STI: ', 21794.09704806807, 'STL: ', 2)
('IDX: ', 4, 'NI: ', 1823.7866455056505, 'STI: ', 1559.5573888843676, 'STL: ', 2)
('IDX: ', 0, 'NI: ', 27483.198193687535, 'STI: ', 23617.88369357372, 'STL: ', 3)
('IDX: ', 1, 'NI: ', 23794.915979009485, 'STI: ', 21794.09704806807, 'STL: ', 2)
